In [2]:
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/kaggle/working/cache
%env HF_DATASETS_CACHE=/kaggle/working/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/kaggle/working/cache
env: HF_DATASETS_CACHE=/kaggle/working/cache
env: CUDA_LAUNCH_BLOCKING=1


## Set Variables

In [1]:
dataset_path = 'fma_filtered'

In [4]:
ignorefiles = ['jazz.00054.wav']

In [5]:
model_name_or_path = "m3hrdadfi/wav2vec2-base-100k-gtzan-music-genres"
# model_name_or_path = "models/wav2vec2-base-100k-gtzan-music-genres"

## Import Libraries

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio

import transformers
from transformers import (
    HfArgumentParser,
    TrainingArguments,
    EvalPrediction,
    AutoConfig,
    Wav2Vec2Processor,
    Wav2Vec2FeatureExtractor,
    is_apex_available,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint, is_main_process


import os
import sys
import librosa
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm

import torchaudio
from sklearn.model_selection import train_test_split


## wav2vec2 model

In [7]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput

In [8]:
@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [9]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

In [10]:
class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

## Prepare Dataset

### Read directory structure

In [11]:
data = []

for path in Path(f'{dataset_path}/genres_original').glob("**/*.wav"):
    if sys.platform == 'win32':
        pathsep = '\\'
    else:
        pathsep = '/'
        
    label = str(path).split(pathsep)[-2]
    name = str(path).split(pathsep)[-1]
    
    if name in ignorefiles:
        continue

    data.append({
        # "name": name,
        "path": path,
        "label": label
    })

In [12]:
df = pd.DataFrame(data)
df.head()

,path,label
0,../input/gtzan-dataset-music-genre-classificat...,disco
1,../input/gtzan-dataset-music-genre-classificat...,disco
2,../input/gtzan-dataset-music-genre-classificat...,disco
3,../input/gtzan-dataset-music-genre-classificat...,disco
4,../input/gtzan-dataset-music-genre-classificat...,disco


In [13]:
df.groupby("label").count()[["path"]]

,path
label,
blues,100
classical,100
country,100
disco,100
hiphop,100
jazz,99
metal,100
pop,100
reggae,100


### Split train and test set

In [14]:
save_path = "/kaggle/working"

train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["label"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)

(799, 2)
(200, 2)


In [15]:
train_df, rest_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["label"])
valid_df, test_df = train_test_split(rest_df, test_size=0.5, random_state=101, stratify=rest_df["label"])

train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", encoding="utf-8", index=False)
valid_df.to_csv(f"{save_path}/valid.csv", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", encoding="utf-8", index=False)


print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

(799, 2)
(100, 2)
(100, 2)


## Prepare Data for Training

In [16]:
from datasets import load_dataset, load_metric

In [17]:
data_files = {
    "train": f"{save_path}/train.csv", 
    "validation": f"{save_path}/test.csv",
    # "test": f"{dataset_path}/test.csv",
}

dataset = load_dataset("csv", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /kaggle/working/cache/csv/default-92f5121365d5c0d7/0.0.0/50d3f5d1aef1732094f95566e0c67506165deea4ff693f78b1a17588fb6bc108. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

In [19]:
# We need to specify the input and output column
input_column = "path"
output_column = "label"

In [20]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes:\n {label_list}")

A classification problem with 10 classes:
 ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']


In [21]:
from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2FeatureExtractor

In [22]:
model_name_or_path = "facebook/wav2vec2-base-100k-voxpopuli"
pooling_mode = "mean"

In [23]:
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

In [24]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
target_sampling_rate = feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

Downloading:   0%|          | 0.00/213 [00:00<?, ?B/s]

The target sampling rate: 16000


## Preprocess Data

In [31]:
label_list

['Electronic',
 'Experimental',
 'Folk',
 'Hip-Hop',
 'Instrumental',
 'International',
 'Pop',
 'Rock']

In [25]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):
    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1
    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]
    result = feature_extractor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)
    return result

In [26]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    # num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    # num_proc=4
)

  0%|          | 0/80 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


  0%|          | 0/10 [00:00<?, ?ba/s]

## Set Up Trainer

In [27]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor, Wav2Vec2FeatureExtractor

In [28]:
@dataclass
class DataCollatorCTCWithPadding:
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.feature_extractor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [29]:
data_collator = DataCollatorCTCWithPadding(feature_extractor=feature_extractor, padding=True)

In [30]:
is_regression = False

In [31]:
import numpy as np
from transformers import EvalPrediction

In [32]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [33]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-100k-voxpopuli were not used when initializing Wav2Vec2ForSpeechClassification: ['project_hid.weight', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.bias', 'quantizer.weight_proj.weight', 'project_q.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-100k-voxpopuli and are newly initialized: ['classifier.out_proj.bias', 

In [34]:
model.freeze_feature_extractor()

In [35]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()


In [36]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/wav2vec2-base-100k-eating-sound-collection",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-base-100k-eating-sound-collection"
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
)

training_args = TrainingArguments(
    # _n_gpu=1,
    adafactor=False,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    bf16=False,
    bf16_full_eval=False,
    dataloader_drop_last=False,
    dataloader_num_workers=0,
    dataloader_pin_memory=True,
    ddp_bucket_cap_mb=None,
    ddp_find_unused_parameters=None,
    debug=[],
    deepspeed=None,
    disable_tqdm=False,
    do_eval=True,
    do_predict=True,
    do_train=True,
    eval_accumulation_steps=None,
    eval_steps=500,
    evaluation_strategy="steps",
    fp16=True,
    fp16_full_eval=False,
    gradient_accumulation_steps=2,
    gradient_checkpointing=False,
    greater_is_better=None,
    group_by_length=False,
    ignore_data_skip=False,
    label_names=None,
    label_smoothing_factor=0.0,
    learning_rate=0.0001,
    load_best_model_at_end=False,
    local_rank=-1,
    max_grad_norm=1.0,
    max_steps=-1,
    metric_for_best_model=None,
    no_cuda=False,
    num_train_epochs=20.0,
    optim="adamw_hf",
    output_dir=f"{save_path}/wav2vec2-base-100k-voxpopuli-gtzan-music",
    overwrite_output_dir=True,
    past_index=-1,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=1,
    prediction_loss_only=False,
    remove_unused_columns=True,
    report_to=['tensorboard'],
    resume_from_checkpoint=None,
    save_on_each_node=False,
    save_steps=100,
    save_total_limit=2,
    seed=42,
    sharded_ddp=[],
    skip_memory_metrics=True,
    tf32=None,
    tpu_metrics_debug=False,
    tpu_num_cores=None,
    use_legacy_prediction_loop=False,
    warmup_ratio=0.0,
    warmup_steps=2000,
    weight_decay=0.0,
    xpu_backend=None,
)

In [37]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=feature_extractor,
)

Using amp half precision backend


## Train

In [38]:
torch.cuda.empty_cache()

In [39]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: path. If path are not expected by `Wav2Vec2ForSpeechClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 799
  Num Epochs = 20
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 7980


Step,Training Loss,Validation Loss,Accuracy
500,2.190000,1.917119,0.300000
1000,1.761900,1.468160,0.540000
1500,1.470900,1.333659,0.610000
2000,1.363400,1.366261,0.600000
2500,1.244800,1.417079,0.620000
3000,1.157800,1.382689,0.660000
3500,1.053400,1.571765,0.640000
4000,0.877100,1.174131,0.780000
4500,0.721400,1.639719,0.770000
5000,0.667100,2.043783,0.700000


Saving model checkpoint to /kaggle/working/wav2vec2-base-100k-voxpopuli-gtzan-music/checkpoint-100
Configuration saved in /kaggle/working/wav2vec2-base-100k-voxpopuli-gtzan-music/checkpoint-100/config.json
Model weights saved in /kaggle/working/wav2vec2-base-100k-voxpopuli-gtzan-music/checkpoint-100/pytorch_model.bin
Feature extractor saved in /kaggle/working/wav2vec2-base-100k-voxpopuli-gtzan-music/checkpoint-100/preprocessor_config.json
/opt/conda/lib/python3.7/site-packages/transformers/trainer.py:1443: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
Saving model checkpoint to /kaggle/working/wav2vec2-base-100k-voxpopuli-gtzan-music/checkpoint-200
Configuration saved in /kaggle/working/wav2vec2-base-100k-

TrainOutput(global_step=7980, training_loss=0.8990356990269253, metrics={'train_runtime': 12430.4336, 'train_samples_per_second': 1.286, 'train_steps_per_second': 0.642, 'total_flos': 4.3734474320939146e+18, 'train_loss': 0.8990356990269253, 'epoch': 20.0})

## Evaluation

In [40]:
import numpy as np
import librosa
import torchaudio
from datasets import load_dataset, load_metric
from sklearn.metrics import classification_report

In [41]:
test_dataset = load_dataset("csv", data_files={"test": "./test.csv"})["test"]
test_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /kaggle/working/cache/csv/default-e2207ec58e9ad0c6/0.0.0/50d3f5d1aef1732094f95566e0c67506165deea4ff693f78b1a17588fb6bc108. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['path', 'label'],
    num_rows: 100
})

In [42]:
model_name_or_path = "m3hrdadfi/wav2vec2-base-100k-gtzan-music-genres"
model_name_or_path = "./wav2vec2-base-100k-voxpopuli-gtzan-music/checkpoint-7900"

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(model_name_or_path)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
sampling_rate = feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

loading configuration file ./wav2vec2-base-100k-voxpopuli-gtzan-music/checkpoint-7900/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "./wav2vec2-base-100k-voxpopuli-gtzan-music/checkpoint-7900",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForSpeechClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extra

In [44]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = feature_extractor(batch["speech"], sampling_rate=feature_extractor.sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)

    with torch.no_grad():
        logits = model(input_values).logits 

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    batch["predicted"] = pred_ids
    return batch

In [45]:
test_dataset = test_dataset.map(speech_file_to_array_fn)

  0%|          | 0/100 [00:00<?, ?ex/s]

In [46]:
result = test_dataset.map(predict, batched=True, batch_size=8)

  0%|          | 0/13 [00:00<?, ?ba/s]

In [47]:
label_names = [config.id2label[i] for i in range(config.num_labels)]

In [48]:
y_true = [config.label2id[name] for name in result["label"]]
y_pred = result["predicted"]

print(y_true[:5])
print(y_pred[:5])

[8, 8, 8, 4, 5]
[8, 8, 8, 4, 4]


In [49]:
print(classification_report(y_true, y_pred, target_names=label_names))

              precision    recall  f1-score   support

       blues       0.82      0.90      0.86        10
   classical       1.00      1.00      1.00        10
     country       0.86      0.60      0.71        10
       disco       0.83      0.50      0.62        10
      hiphop       0.67      0.80      0.73        10
        jazz       1.00      0.80      0.89        10
       metal       0.64      0.90      0.75        10
         pop       1.00      0.50      0.67        10
      reggae       0.47      0.90      0.62        10
        rock       0.50      0.40      0.44        10

    accuracy                           0.73       100
   macro avg       0.78      0.73      0.73       100
weighted avg       0.78      0.73      0.73       100



## Clean Up

In [50]:
!rm -rf cache

## EK

In [17]:
import torchaudio

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# Define the genres and their corresponding folders
genres = ['Hip-Hop', 'Rock', 'Pop', 'Folk', 'Experimental', 'Electronic', 'Instrumental', 'International']
data_path = 'fma_filtered'  # Path to your data folder

# Create a list to store the data
data = []

# Loop over each genre folder
for genre in genres:
    genre_folder = os.path.join(data_path, genre)
    
    # Get the audio files in the genre folder
    audio_files = os.listdir(genre_folder)
    
    # Sample 100 files from each genre
    audio_files = audio_files[:100]
    
    # Add the data to the list
    for audio_file in audio_files:
        audio_path = os.path.join(genre_folder, audio_file)
        data.append({
            'path': audio_path,
            'label': genre
        })

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df['label'])

# Save the train and test sets as CSV files
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

In [2]:
# Load the dataset using the Hugging Face datasets library
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation': 'test.csv'}, cache_dir=False)

Using custom data configuration default-a7e4f04e7fd397ab


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/eeshakhanna/.cache/huggingface/datasets/csv/default-a7e4f04e7fd397ab/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Get the train and validation datasets
train_dataset = dataset['train']
valid_dataset = dataset['validation']

# Preprocess the data
def preprocess_function(examples):
    # Perform your preprocessing steps here
    return examples

train_dataset = train_dataset.map(preprocess_function)
valid_dataset = valid_dataset.map(preprocess_function)

  0%|          | 0/640 [00:00<?, ?ex/s]

  0%|          | 0/160 [00:00<?, ?ex/s]

In [11]:
# We need to specify the input and output column
input_column = "path"
output_column = "label"

In [12]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes:\n {label_list}")

A classification problem with 8 classes:
 ['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']


In [7]:
from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2FeatureExtractor

In [8]:
model_name_or_path = "facebook/wav2vec2-base-100k-voxpopuli"
pooling_mode = "mean"

In [13]:
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

In [14]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
target_sampling_rate = feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

The target sampling rate: 16000


In [32]:
label_list

['Electronic',
 'Experimental',
 'Folk',
 'Hip-Hop',
 'Instrumental',
 'International',
 'Pop',
 'Rock']

In [35]:
def speech_file_to_array_fn(path):    
    try:
        waveform, sr = torchaudio.load(audio_file)
        waveform = waveform.mean(dim=0, keepdim=True)  # Convert to mono by averaging the channels
        if sr != target_sr:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
            waveform = resampler(waveform)
        if waveform.size(1) > max_length * target_sr:
            waveform = waveform[:, : max_length * target_sr]
    except Exception as e:
        print(f"Error loading audio file at {path}: {e}")
        return None
    return waveform

def label_to_id(label, label_list):
    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1
    return label


def preprocess_function(examples):
    speech_list = []
    target_list = []
    for path, label in zip(examples[input_column], examples[output_column]):
        print('print',path,label)
        speech = speech_file_to_array_fn(path)
        if speech is not None:
            speech_list.append(speech)
            target_list.append(label_to_id(label, label_list))

    result = feature_extractor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = target_list

    return result

In [49]:
def load_preprocessed(examples):
    speech_list = []
    target_list = []
    for path, label in zip(examples[input_column], examples[output_column]):
        #print(path)
        print("path ek", os.path.join("preprocesses_tensors_full/", path.split("/")[2]  + '.pt'))
        speech = os.path.join("preprocesses_tensors_full/", path.split("/")[2]  + '.pt')
        if speech:
            speech_list.append(speech)
            target_list.append(label_to_id(label, label_list))

In [51]:
train_dataset = train_dataset.map(
    load_preprocessed,
    batch_size=10,
    batched=True,
    # num_proc=4
)
eval_dataset = valid_dataset.map(
    load_preprocessed,
    batch_size=10,
    batched=True,
    # num_proc=4
)

  0%|          | 0/64 [00:00<?, ?ba/s]

path ek preprocesses_tensors_full/0758_Electronic.mp3.pt
path ek preprocesses_tensors_full/0509_Hip-Hop.mp3.pt
path ek preprocesses_tensors_full/0928_International.mp3.pt
path ek preprocesses_tensors_full/0305_Experimental.mp3.pt
path ek preprocesses_tensors_full/0282_Rock.mp3.pt
path ek preprocesses_tensors_full/0328_Electronic.mp3.pt
path ek preprocesses_tensors_full/0332_Electronic.mp3.pt
path ek preprocesses_tensors_full/0688_Hip-Hop.mp3.pt
path ek preprocesses_tensors_full/0051_Folk.mp3.pt
path ek preprocesses_tensors_full/0729_Pop.mp3.pt
path ek preprocesses_tensors_full/0511_Pop.mp3.pt
path ek preprocesses_tensors_full/0732_Rock.mp3.pt
path ek preprocesses_tensors_full/0470_Electronic.mp3.pt
path ek preprocesses_tensors_full/0649_Electronic.mp3.pt
path ek preprocesses_tensors_full/0683_Pop.mp3.pt
path ek preprocesses_tensors_full/0845_Pop.mp3.pt
path ek preprocesses_tensors_full/0339_Hip-Hop.mp3.pt
path ek preprocesses_tensors_full/0805_Folk.mp3.pt
path ek preprocesses_tensors_f

  0%|          | 0/16 [00:00<?, ?ba/s]

path ek preprocesses_tensors_full/0969_Experimental.mp3.pt
path ek preprocesses_tensors_full/0211_Rock.mp3.pt
path ek preprocesses_tensors_full/0884_Instrumental.mp3.pt
path ek preprocesses_tensors_full/0133_Experimental.mp3.pt
path ek preprocesses_tensors_full/0115_Instrumental.mp3.pt
path ek preprocesses_tensors_full/0378_Instrumental.mp3.pt
path ek preprocesses_tensors_full/0963_Rock.mp3.pt
path ek preprocesses_tensors_full/0089_Electronic.mp3.pt
path ek preprocesses_tensors_full/0672_Rock.mp3.pt
path ek preprocesses_tensors_full/0495_Electronic.mp3.pt
path ek preprocesses_tensors_full/0706_International.mp3.pt
path ek preprocesses_tensors_full/0435_Instrumental.mp3.pt
path ek preprocesses_tensors_full/0239_Electronic.mp3.pt
path ek preprocesses_tensors_full/0720_Pop.mp3.pt
path ek preprocesses_tensors_full/0188_Folk.mp3.pt
path ek preprocesses_tensors_full/0766_Experimental.mp3.pt
path ek preprocesses_tensors_full/0462_Electronic.mp3.pt
path ek preprocesses_tensors_full/0715_Folk.m